In [3]:
import pandas as pd
import requests
import json
import html
import datetime
import csv
import re


In [4]:
companies_df = pd.read_csv('../lever_companies.csv')
companies_df


,Company,Token
0,Anduril Industries,anduril
1,Angle Health,AngleHealth
2,Anyscale,anyscale
3,Ascend.io,ascend.io
4,Asics,?asics?
...,...,...
57,Up&Up,Up&Up
58,Veeva,veeva
59,Xage Security,xage-security
60,YugabyteDB,yugabyte


In [5]:
companies_dict = {}
for _ , row in companies_df.iterrows():
  companies_dict[row['Token']] = row['Company']


companies_dict


{'anduril': 'Anduril Industries',
 'AngleHealth': 'Angle Health',
 'anyscale': 'Anyscale',
 'ascend.io': 'Ascend.io',
 '?asics?': 'Asics',
 ' atlassian': 'Atlassian',
 'aurorasolar': 'Aurora Solar',
 'automox': 'Automox',
 'axoni': 'Axoni',
 '?belvedereTrading?': 'Belvedere Trading',
 'bluebottlecoffee': 'Blue Bottle Coffee',
 'bodyport': 'Bodyport',
 'boomandbucket': 'Boom & Bucket',
 '?butcherbox?': 'Butcher Box',
 '?captiv8?': 'Captiv8',
 'cellares': 'Cellares',
 'cohere': 'Cohere',
 'confluent': 'Confluent',
 'credinvestments': 'CRED Investments',
 'eargo': 'Eargo',
 ' facta': 'Facta',
 'goforward': 'Forward',
 'front': 'Front',
 'getlabs': 'Getlabs',
 'gopuff': 'Gopuff',
 '?greenthumbindustries?': 'Green Thumb industr.',
 'hihello': 'HiHello',
 'hive': 'Hive',
 'learnwithjuni': 'Juni Learning',
 'leolabs-2': 'LeoLabs',
 'logrocket': 'LogRocket',
 'MachinaLabs': 'Machina Labs',
 'nava': 'Nava',
 'neeva': 'Neeva',
 'netflix': 'Netflix',
 'nexient': 'Nexient',
 'observeinc': 'Observe

In [6]:
tokens = set(companies_df.Token)
companies_clean = companies_dict
companies_bad = {}

In [7]:
len(companies_dict.keys())

62

In [8]:
for token in tokens:
    if not requests.get(f'https://jobs.lever.co/v0/postings/{token}?mode=json'):
      print(f'API call for {token} failed')
      companies_bad[token] = companies_dict.get(token)
      companies_clean.pop(token)

API call for ?asics? failed
API call for ?belvedereTrading? failed
API call for ?captiv8? failed
API call for ?perpay? failed
API call for  sure failed
API call for  facta failed
API call for ?butcherbox? failed
API call for  Up&Up failed
API call for  translifeline failed
API call for ?greenthumbindustries? failed
API call for starburst failed
API call for spotify  failed
API call for boomandbucket failed
API call for front failed
API call for  atlassian failed


In [7]:
companies_bad

{'?sure?': 'Sure',
 'boomandbucket': 'Boom & Bucket',
 '?starbust?': 'Starbusrt',
 'front': 'Front',
 '?butcherbox?': 'Butcher Box',
 '?asics?': 'Asics',
 '?atlassian?': 'Atlassian',
 '?Up&Up?': 'Up&Up',
 '?captiv8?': 'Captiv8',
 '?facta?': 'Facta',
 '?belvedereTrading?': 'Belvedere Trading',
 '?translifeline?': 'TransLifeline',
 '?greenthumbindustries?': 'Green Thumb industr.',
 '?spotify? ': 'Spotify',
 '?perpay?': 'Perpay'}

In [8]:
roles = {'developer','engineer', 'data', 'engineering', 'frontend', 'software', 'apprentice', 'analyst' , 'quality', 'apprenticeship', 'front-end', 'backend', 'back-end', 'jr.', 'jr' }
# Includes 'software' for titles that are just 'software enginer', etc.
levels = {'junior', 'entry-level', 'grad', 'graduate', 'data', 'apprentice', 'apprenticeship', 'software', 'qa', 'quality', 'test', 'entry', 'intern', 'i', '1', 'associate', 'co-op'}
# Optional, but helps exclude higher level postiions
exclude = {'senior', 'principal' , 'sr.', 'sr' 'ii', 'iii', 'director' }



In [9]:
res=requests.get(f'https://jobs.lever.co/v0/postings/tegus?mode=json')
jobs=json.loads(res.text)
job = jobs[0]

job.keys()

dict_keys(['additionalPlain', 'additional', 'categories', 'createdAt', 'descriptionPlain', 'description', 'id', 'lists', 'text', 'country', 'workplaceType', 'hostedUrl', 'applyUrl'])

In [10]:
job['categories']

{'commitment': 'Full-Time',
 'department': 'Operations',
 'location': 'Chicago, IL',
 'team': 'Operations'}

In [11]:
print(datetime.datetime.fromtimestamp(job.get('createdAt')/1000.).isoformat())


2020-12-10T18:18:24.613000


In [12]:
results = []

for token in tokens:
  res=requests.get(f'https://jobs.lever.co/v0/postings/{token}?mode=json')
  if res:
    jobs=json.loads(res.text)

  company= companies_clean.get(token)

  for job in jobs:
    title = set(job.get('text').lower().split())

    if title.intersection(roles) and title.intersection(levels) and not title.intersection(exclude):
      job_info = {'title': job.get('text'), 'company' : company, 'description': job.get('descriptionPlain'),
                'link' : job.get('applyUrl'), 'remote' : True if job.get('workplaceType').lower() == 'remote' else False, 'location' : job.get('categories').get('location') ,  'date_posted': datetime.datetime.fromtimestamp(job.get('createdAt')/1000.).isoformat()} 
      
      results.append(job_info)



NameError: name 'roles' is not defined

In [13]:
df = pd.DataFrame()
df = df.from_records(results)
df

,title,company,description,link,remote,location,date_posted
0,Data Engineer,Angle Health,"Changing Healthcare For Good\nAt Angle Health,...",https://jobs.lever.co/AngleHealth/44c7ba3b-aa0...,True,Remote,2023-02-14T00:41:59.947000
1,Deployed Software Engineer,Angle Health,"Changing Healthcare For Good\nAt Angle Health,...",https://jobs.lever.co/AngleHealth/cb485eee-945...,True,Remote,2023-01-12T15:14:16.702000
2,"Software Engineer, Backend",Angle Health,"Changing Healthcare For Good\nAt Angle Health,...",https://jobs.lever.co/AngleHealth/2bbcdb54-548...,True,Remote,2021-11-23T15:44:01.367000
3,"Software Engineer, Full Stack",Angle Health,"Changing Healthcare For Good\nAt Angle Health,...",https://jobs.lever.co/AngleHealth/04b36f5f-a1c...,True,Remote,2023-02-14T00:50:36.865000
4,"Software Engineer, Strategy",Angle Health,"Changing Healthcare For Good\nAt Angle Health,...",https://jobs.lever.co/AngleHealth/b2d736ea-adc...,True,Remote,2023-02-14T21:16:18.955000
...,...,...,...,...,...,...,...
874,Test Automation Engineer - Networking,Zoox,"As a test automation engineer in networking, y...",https://jobs.lever.co/zoox/48f45666-cde1-4599-...,False,"Foster City, CA",2023-01-03T22:34:35.424000
875,Test Infrastructure Engineer,Zoox,Zoox is looking for a talented Test Infrastruc...,https://jobs.lever.co/zoox/6d3248d6-b4bd-4d73-...,False,"San Carlos, CA",2022-04-28T15:59:44.862000
876,"Software Engineer, Computational Scientist",Aurora Solar,About the role:\n\nAurora Solar is a fast-grow...,https://jobs.lever.co/aurorasolar/62e5ad48-559...,True,Canada,2022-09-01T17:31:01.840000
877,"Software Engineer, Data and GIS",Aurora Solar,About the role:\n\nAurora Solar is a fast-grow...,https://jobs.lever.co/aurorasolar/025beacf-d90...,True,Canada,2021-01-27T19:28:52.028000


In [14]:
df.sort_values(by='date_posted', ascending=False)

,title,company,description,link,remote,location,date_posted
552,Software Engineers,OtterTune,"Job Location:\n201 S Craig Street, Suite 2E\nP...",https://jobs.lever.co/ottertune/d8ed61a5-f486-...,True,"Pittsburgh, PA",2023-05-03T11:54:37.754000
614,Software Engineer - Data Systems,Anduril Industries,Anduril Industries is a defense technology com...,https://jobs.lever.co/anduril/5b3dd311-1db1-49...,False,"Seattle, WA",2023-05-01T17:20:22.145000
702,Software Engineer - Data Systems,None,Anduril Industries is a defense technology com...,https://jobs.lever.co/anduril/5b3dd311-1db1-49...,False,"Seattle, WA",2023-05-01T17:20:22.145000
637,Staff Software Engineer,Anduril Industries,Anduril Industries is a defense technology com...,https://jobs.lever.co/anduril/3219978b-ab6e-42...,False,"Seattle, WA",2023-05-01T17:06:45.996000
725,Staff Software Engineer,None,Anduril Industries is a defense technology com...,https://jobs.lever.co/anduril/3219978b-ab6e-42...,False,"Seattle, WA",2023-05-01T17:06:45.996000
...,...,...,...,...,...,...,...
577,Frontend Software Engineer,Anduril Industries,Anduril Industries is a defense technology com...,https://jobs.lever.co/anduril/a79b3c52-aaf8-47...,False,"Costa Mesa, CA",2018-07-17T18:19:39.522000
873,Technical Program Manager - Software,Zoox,Zoox has embarked on a highly ambitious journe...,https://jobs.lever.co/zoox/b5b5bb0c-1f95-446e-...,False,"Foster City, CA",2016-05-17T13:37:06.372000
799,Senior/Staff Software Engineer - Machine Learning,Zoox,"At Zoox, you will collaborate with a team of w...",https://jobs.lever.co/zoox/81f7fbe7-4fd3-4814-...,False,"Foster City, CA",2016-02-11T20:45:48.453000
832,Software Development Engineer in Test,Zoox,"Autonomous vehicles have some of the largest, ...",https://jobs.lever.co/zoox/206cbbe7-f2ef-40f1-...,False,"Foster City, CA",2016-02-11T17:06:53.811000
